In [77]:
import torch
from transformers import BertTokenizer
#from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

#clear_output()
print("PyTorch 版本：", torch.__version__)
#====================================================
vocab = tokenizer.vocab
print("字典大小：", len(vocab))
#====================================================
text = "[CLS] 等到潮水 [MASK] 了，就知道誰沒穿褲子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

PyTorch 版本： 1.3.1+cu92
字典大小： 21128


In [78]:
import os
import pandas as pd



# 簡單的數據清理，去除空白標題的 examples
df_train = pd.read_csv("train.csv")
empty_title = ((df_train['title2_zh'].isnull()) \
               | (df_train['title1_zh'].isnull()) \
               | (df_train['title2_zh'] == '') \
               | (df_train['title2_zh'] == '0'))
df_train = df_train[~empty_title]

# 剔除過長的樣本以避免 BERT 無法將整個輸入序列放入記憶體不多的 GPU
MAX_LENGTH = 30
df_train = df_train[~(df_train.title1_zh.apply(lambda x : len(x)) > MAX_LENGTH)]
df_train = df_train[~(df_train.title2_zh.apply(lambda x : len(x)) > MAX_LENGTH)]

# 只用 1% 訓練數據看看 BERT 對少量標註數據有多少幫助
SAMPLE_FRAC = 0.01
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9527)

# 去除不必要的欄位並重新命名兩標題的欄位名
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['title1_zh', 'title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']
#df_train = df_train.iloc[0:1]
#df_train.loc[0][1] = '儘管中國宣稱2019 新型冠狀病毒（COVID-19）疫情在境內受到控制，但中國官方和CBA聯盟不同調，複賽仍遙遙無期，加上中國又頒布外國人禁令，林書豪等外援提早返中恐都是白跑一趟。從疫情爆發後CBA自1月開始停賽，日前宣布將在4月15日重啟，還曾傳出外援包含林書豪等人若不歸隊，將面臨終身禁賽等處分，逼迫外援們提早返中進行14天隔離檢疫，以利備戰複賽。林書豪在3月19日於社群媒體PO出照片，證明他已經返抵中國，就在一切蓄勢待發下，中國國家體育總局卻在3月31日公告暫緩馬拉松等體育賽事，理由為貫徹「外防輸入、內防反彈」控制疫情。先前3月28日，中國官方宣布旅遊禁令，暫停外國人入境，多隊外援和外籍教頭可能無法如期出賽，CBA本來和各隊取得初步共識，以全本土開戰，但如今當局下令不准打，什麼方案都是白談了，外援提早入境恐怕也都是白跑一趟，目前何時複賽仍未明朗。	儘管中國宣稱2019 新型冠狀病毒（COVID-19）疫情在境內受到控制，但中國官方和CBA聯盟不同調，複賽仍遙遙無期，加上中國又頒布外國人禁令，林書豪等外援提早返中恐都是白跑一趟。從疫情爆發後CBA自1月開始停賽，日前宣布將在4月15日重啟，還曾傳出外援包含林書豪等人若不歸隊，將面臨終身禁賽等處分，逼迫外援們提早返中進行14天隔離檢疫，以利備戰複賽。林書豪在3月19日於社群媒體PO出照片，證明他已經返抵中國，就在一切蓄勢待發下，中國國家體育總局卻在3月31日公告暫緩馬拉松等體育賽事，理由為貫徹「外防輸入、內防反彈」控制疫情。先前3月28日，中國官方宣布旅遊禁令，暫停外國人入境，多隊外援和外籍教頭可能無法如期出賽，CBA本來和各隊取得初步共識，以全本土開戰，但如今當局下令不准打，什麼方案都是白談了，外援提早入境恐怕也都是白跑一趟，目前何時複賽仍未明朗。'
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用
df_train.to_csv("train.tsv", sep="\t", index=False)

print("訓練樣本數：", len(df_train))
df_train.head()

訓練樣本數： 2657


,text_a,text_b,label
0,苏有朋要结婚了，但网友觉得他还是和林心如比较合适,好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！,unrelated
1,爆料李小璐要成前妻了贾乃亮模仿王宝强一步到位、快刀斩乱麻！,李小璐要变前妻了？贾乃亮可能效仿王宝强当机立断，快刀斩乱麻！,agreed
2,为彩礼，母亲把女儿嫁给陌生男子，十年后再见面，母亲湿了眼眶,阿姨，不要彩礼是觉得你家穷，给你台阶下，不要以为我嫁不出去！,unrelated
3,猪油是个宝，一勺猪油等于十副药，先备起来再说,传承千百的猪油为何变得人人唯恐避之不及？揭开猪油的四大谣言！,unrelated
4,剖析：香椿，为什么会致癌？,香椿含亚硝酸盐多吃会致癌？测完发现是谣言,disagreed


In [79]:
df_test = pd.read_csv("test.csv")
df_test = df_test.loc[:, ["title1_zh", "title2_zh", "id"]]
df_test.columns = ["text_a", "text_b", "Id"]
df_test.to_csv("test.tsv", sep="\t", index=False)

print("預測樣本數：", len(df_test))
df_test.head()

預測樣本數： 80126


,text_a,text_b,Id
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,321187
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,321190
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,321189
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,321193
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,321191


In [80]:
from torch.utils.data import Dataset
import pysnooper
    
class FakeNewsDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {'agreed': 0, 'disagreed': 1, 'unrelated': 2}
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    @pysnooper.snoop()
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, text_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        # 第二個句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len
    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = FakeNewsDataset("train", tokenizer=tokenizer)

In [81]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
text_a, text_b, label = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{text_a}
句子 2：{text_b}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
句子 1：苏有朋要结婚了，但网友觉得他还是和林心如比较合适
句子 2：好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！
分類  ：unrelated

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101, 5722, 3300, 3301, 6206, 5310, 2042,  749, 8024,  852, 5381, 1351,
        6230, 2533,  800, 6820, 3221, 1469, 3360, 2552, 1963, 3683, 6772, 1394,
        6844,  102, 1962, 7318, 6057, 5310, 2042, 5314,  679, 2042, 3184, 4638,
        4912, 2269, 2803, 5709, 4413, 8024,  948, 7450, 4638, 4912, 2269, 2957,
        3717, 7027, 5010, 1526, 5722, 3300, 3301, 8013,  102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1])

label_tensor   ：2

--------------------

[還原 tokens_tensors]
[CLS]苏有朋要结婚了，但网友觉得他还是和林心如比较合适[SEP]好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！[SEP]



Source path:... <ipython-input-80-ed1903d6133d>
Starting var:.. self = <__main__.FakeNewsDataset object at 0x0000019E7AD14D08>
Starting var:.. idx = 0
13:52:07.250323 call        16     def __getitem__(self, idx):
13:52:07.250323 line        17         if self.mode == "test":
13:52:07.250323 line        21             text_a, text_b, label = self.df.iloc[idx, :].values
New var:....... text_a = '苏有朋要结婚了，但网友觉得他还是和林心如比较合适'
New var:....... text_b = '好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！'
New var:....... label = 'unrelated'
13:52:07.251321 line        23             label_id = self.label_map[label]
New var:....... label_id = 2
13:52:07.251321 line        24             label_tensor = torch.tensor(label_id)
New var:....... label_tensor = tensor(2)
13:52:07.251321 line        27         word_pieces = ["[CLS]"]
New var:....... word_pieces = ['[CLS]']
13:52:07.251321 line        28         tokens_a = self.tokenizer.tokenize(text_a)
New var:....... tokens_a = ['苏', '有', '朋', '要', '结', '婚', '了', '，', '但

In [82]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 1
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch)

In [83]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([1, 57]) 
tensor([[ 101, 5722, 3300, 3301, 6206, 5310, 2042,  749, 8024,  852, 5381, 1351,
         6230, 2533,  800, 6820, 3221, 1469, 3360, 2552, 1963, 3683, 6772, 1394,
         6844,  102, 1962, 7318, 6057, 5310, 2042, 5314,  679, 2042, 3184, 4638,
         4912, 2269, 2803, 5709, 4413, 8024,  948, 7450, 4638, 4912, 2269, 2957,
         3717, 7027, 5010, 1526, 5722, 3300, 3301, 8013,  102]])
------------------------
segments_tensors.shape = torch.Size([1, 57])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])
------------------------
masks_tensors.shape    = torch.Size([1, 57])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])
------------------------

Starting var:.. self = <__main__.FakeNewsDataset object at 0x0000019E7AD14D08>
Starting var:.. idx = 0
13:52:07.279259 call        16     def __getitem__(self, idx):
13:52:07.280243 line        17         if self.mode == "test":
13:52:07.280243 line        21             text_a, text_b, label = self.df.iloc[idx, :].values
New var:....... text_a = '苏有朋要结婚了，但网友觉得他还是和林心如比较合适'
New var:....... text_b = '好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！'
New var:....... label = 'unrelated'
13:52:07.281240 line        23             label_id = self.label_map[label]
New var:....... label_id = 2
13:52:07.282238 line        24             label_tensor = torch.tensor(label_id)
New var:....... label_tensor = tensor(2)
13:52:07.282238 line        27         word_pieces = ["[CLS]"]
New var:....... word_pieces = ['[CLS]']
13:52:07.282238 line        28         tokens_a = self.tokenizer.tokenize(text_a)
New var:....... tokens_a = ['苏', '有', '朋', '要', '结', '婚', '了', '，', '但', '网...'还', '是', '和', '林', '心', '如', '比', '较', 

In [84]:
# 載入一個可以做中文多分類任務的模型，n_class = 3
from transformers import BertForSequenceClassification, BertModel

PRETRAINED_MODEL_NAME = "bert-base-chinese"



model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)


# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))
        



name            module
----------------------
embeddings      BertEmbeddings(
  (word_embeddings): Embedding(21128, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
encoder         BertEncoder(
  (layer): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )

In [85]:
from transformers import BertModel, BertConfig

# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig(hidden_size=768)

# Initializing a model from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
#configuration = model.config

In [86]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            print('data : ',data)
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            print(outputs)
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            return outputs
            # 用來計算訓練集的分類準確率
            
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
#model = model.to(device)
acc = get_predictions(model, trainloader, compute_acc=True)
#print("classification acc:", acc)

device: cuda:0
data :  (tensor([[ 101, 5722, 3300, 3301, 6206, 5310, 2042,  749, 8024,  852, 5381, 1351,
         6230, 2533,  800, 6820, 3221, 1469, 3360, 2552, 1963, 3683, 6772, 1394,
         6844,  102, 1962, 7318, 6057, 5310, 2042, 5314,  679, 2042, 3184, 4638,
         4912, 2269, 2803, 5709, 4413, 8024,  948, 7450, 4638, 4912, 2269, 2957,
         3717, 7027, 5010, 1526, 5722, 3300, 3301, 8013,  102]]), tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]]), tensor([2]))
(tensor([[[-0.9314,  1.7674, -0.2113,  ..., -0.1514, -0.7655,  0.4663],
         [ 0.2299,  2.0348,  0.1942,  ..., -0.5772, -1.0472,  0.6396],
         [-0.1634,  1.5729, 

Starting var:.. self = <__main__.FakeNewsDataset object at 0x0000019E7AD14D08>
Starting var:.. idx = 0
13:52:12.080857 call        16     def __getitem__(self, idx):
13:52:12.081873 line        17         if self.mode == "test":
13:52:12.081873 line        21             text_a, text_b, label = self.df.iloc[idx, :].values
New var:....... text_a = '苏有朋要结婚了，但网友觉得他还是和林心如比较合适'
New var:....... text_b = '好闺蜜结婚给不婚族的秦岚扔花球，倒霉的秦岚掉水里笑哭苏有朋！'
New var:....... label = 'unrelated'
13:52:12.081873 line        23             label_id = self.label_map[label]
New var:....... label_id = 2
13:52:12.081873 line        24             label_tensor = torch.tensor(label_id)
New var:....... label_tensor = tensor(2)
13:52:12.081873 line        27         word_pieces = ["[CLS]"]
New var:....... word_pieces = ['[CLS]']
13:52:12.082852 line        28         tokens_a = self.tokenizer.tokenize(text_a)
New var:....... tokens_a = ['苏', '有', '朋', '要', '结', '婚', '了', '，', '但', '网...'还', '是', '和', '林', '心', '如', '比', '较', 

In [87]:
acc[0][0][56]

tensor([ 7.6800e-01,  3.4254e-02, -1.2727e+00, -1.6715e+00,  1.3263e+00,
         1.5190e+00, -8.8432e-01,  6.5781e-01,  1.8409e+00,  8.7262e-01,
         5.3732e-01,  6.1185e-01, -3.5615e-01, -5.1365e-02,  2.2069e+00,
         1.6405e+00,  3.2211e-01,  9.5048e-01,  2.1713e+00, -7.3055e-01,
        -1.0986e+00, -1.9607e-01, -6.7902e-02, -8.4460e-02,  2.4786e-03,
         1.7666e+00, -1.0979e+00,  1.4223e+00,  6.2535e-01,  1.8184e+00,
         1.3699e+00,  1.3204e-01,  8.5773e-02, -9.0599e-01,  1.6741e-01,
        -7.2120e-02, -1.7546e+00, -1.0835e+00, -1.7812e-02, -7.7225e-01,
         1.6390e+00, -1.9652e+00, -2.6975e+00, -8.4246e-02,  4.9995e-01,
         9.7035e-01,  5.6937e-01,  2.3565e+00, -6.5288e-01,  7.8862e-01,
         1.3754e-01, -6.8679e-02, -1.0387e+00,  7.9469e-01, -8.5058e-01,
         1.1181e+00,  1.5910e+00, -2.0745e+00,  3.6740e-01, -1.8472e+00,
        -4.9975e-01,  8.8842e-01, -7.7970e-01,  4.2002e-01,  4.0560e-01,
         1.1857e+00,  1.0728e+00,  5.2458e-01, -9.1

In [88]:
acc[0][0][25]

tensor([ 3.3511e-01,  9.7620e-01,  1.6881e-01, -1.6746e-01,  1.7394e+00,
         4.1351e-01, -9.3527e-01, -1.4215e+00,  2.0797e-01,  4.3854e-01,
         1.1743e+00,  2.0035e-02, -7.7253e-02,  3.6862e-02,  9.1399e-01,
         4.8420e-01,  2.5521e-01,  2.4548e-01,  8.0901e-01, -5.5270e-01,
        -1.2932e+00,  1.1728e+00,  1.3104e+00, -6.4525e-04,  1.2407e+00,
         1.5431e+00,  6.0450e-01, -3.5695e-01,  3.2148e-01,  8.8375e-01,
         5.8738e-01,  3.5735e-01,  5.8643e-01,  7.0485e-01,  1.5429e+00,
        -1.3873e+00, -3.6883e-01, -1.8330e+00,  9.5843e-01,  4.2259e-01,
         8.2388e-01,  1.3947e-01, -1.2895e+00,  5.2436e-01, -1.4023e-01,
        -1.1777e+00,  1.1356e+00,  1.7126e+00, -2.7257e-01,  6.3837e-01,
         1.4524e+00, -4.6587e-01, -8.1866e-01,  7.1335e-01,  1.0268e+00,
        -4.3458e-01,  1.1199e+00, -1.0428e+00, -1.6140e+00, -2.9325e-01,
         1.3460e+00,  3.6872e-01, -4.7892e-01, -5.7957e-01, -5.4396e-01,
         1.0506e+00, -4.6336e-01,  3.5428e-01,  6.5

In [89]:
acc[0].shape

torch.Size([1, 57, 768])

In [90]:
acc[1]

tensor([[ 3.5098e-01, -7.9196e-01, -2.1698e-01, -3.6640e-01,  1.5158e-01,
         -1.5165e-01, -1.0893e-01, -1.3903e-01, -5.2109e-01, -6.1910e-01,
         -8.2561e-01,  4.2614e-01,  6.9227e-01,  2.9236e-01,  1.8154e-01,
          7.4684e-02, -6.9392e-01, -6.1038e-01,  4.2819e-01, -3.6064e-01,
          7.5615e-01, -1.5883e-01,  8.2739e-01,  7.6967e-02, -5.1764e-01,
         -5.2033e-01,  5.8470e-01,  3.2987e-01,  5.3080e-02, -6.0996e-01,
          7.0961e-02, -2.9388e-01,  5.0768e-01, -5.1933e-01, -7.2867e-01,
         -9.4709e-02, -6.2928e-01, -5.6185e-01,  1.2785e-01, -5.2781e-01,
          8.2436e-01,  8.9368e-01,  7.9929e-01, -3.8593e-01, -2.2878e-01,
          5.1985e-01, -1.5469e-01, -5.7444e-01, -5.4170e-01, -4.2583e-01,
         -6.7562e-01,  5.3583e-01, -6.9473e-01, -7.5306e-02, -3.6913e-01,
          2.6101e-01,  4.1753e-01,  1.9410e-01, -1.9219e-01,  3.2181e-01,
         -3.3338e-01,  2.8422e-01, -1.8168e-02, -6.1339e-01,  3.0172e-01,
         -7.9172e-02, -3.7844e-01,  4.